Notebook for filtering the accounts and indicators that will be used for selecting stocks for the Magic Formula

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 20
pd.options.display.max_rows = 4

In [2]:
# Lendo a base diretamente do S3 -> são mais de 10 milhões de linhas contábeis!
df = pd.read_feather("s3://aq-dl/FinancialStatements/base.feather")
df

,cia_id,cia_nome,doc_id,doc_arq,doc_tp,doc_ver,doc_env,doc_ref,per_ini,per_fim,per_ref,dem_tp,conta_id,conta_fixa,conta_desc,conta_valor
0,3,CIA MODELO,54947,0000032016033130...,ITR,1,2016-04-02 12:04:12,2016-03-31,2016-01-01,2016-03-31,0,IND,3.01,1,Receita de Venda...,100000000.0
1,3,CIA MODELO,54947,0000032016033130...,ITR,1,2016-04-02 12:04:12,2016-03-31,2016-01-01,2016-03-31,0,IND,3.02,1,Custo dos Bens e...,-10000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10526336,26310,CIA. SECURITIZAD...,108527,0263102021063030...,ITR,1,2021-10-08 15:53:04,2021-06-30,2021-01-01,2021-06-30,0,IND,9.01.01,1,Ações Ordinárias...,5000.0
10526337,26310,CIA. SECURITIZAD...,108527,0263102021063030...,ITR,1,2021-10-08 15:53:04,2021-06-30,2021-01-01,2021-06-30,0,IND,9.01.03,1,Total de Ações (...,5000.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10526338 entries, 0 to 10526337
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   cia_id       category      
 1   cia_nome     category      
 2   doc_id       category      
 3   doc_arq      category      
 4   doc_tp       category      
 5   doc_ver      category      
 6   doc_env      datetime64[ns]
 7   doc_ref      datetime64[ns]
 8   per_ini      datetime64[ns]
 9   per_fim      datetime64[ns]
 10  per_ref      category      
 11  dem_tp       category      
 12  conta_id     category      
 13  conta_fixa   category      
 14  conta_desc   category      
 15  conta_valor  float64       
dtypes: category(11), datetime64[ns](4), float64(1)
memory usage: 640.0 MB


In [4]:
# Versão inicial: somente dados auditados serão usados -> (DFP) 
df.query('doc_tp == "DFP"', inplace=True)
# Remover colunas que não serão usadas no backtesting
# doc_ref -> já está implícito no período inicial e no final
# doc_ver e doc_id -> o controle da versão do documento será feito pela horário
# de envio e pelo empresa
df.drop(columns=["doc_tp", "doc_arq", "doc_ref", "doc_ver", "doc_id"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,per_ref,dem_tp,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,IND,1,1,Ativo Total,184750000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,CON,1,1,Ativo Total,204561000.0
...,...,...,...,...,...,...,...,...,...,...,...
7035845,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,0,IND,9.02.02,1,Ações Preferenci...,51107.0
7035846,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,0,IND,9.02.03,1,Total de Ações (...,51107.0


In [5]:
# Iremos simular somente com dados consolidados (CON)
# Os dados com o número de ações foram estão no balanço individual (IND)
# A conta que interessa é a "9.01.03" -> "Total de Ações (Capital Integralizado)"
df.query('dem_tp == "CON" or conta_id == "9.01.03"', inplace=True)
# Remover coluna que não será mais usada no backtesting
df.drop(columns=["dem_tp"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,per_ref,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,1,1,Ativo Total,204561000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,1.01,1,Ativo Circulante,169771000.0
...,...,...,...,...,...,...,...,...,...,...
2905266,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,0,7.09.04.04,1,Part. Não Contro...,2499000.0
2905267,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,0,9.01.03,1,Total de Ações (...,583232411.0


* (pág. 138) *For purposes of the study, earnings-related numbers were based on the latest 12-month period, balance sheet items were based on the most recent balance sheet, and market prices were based on the most recent closing price. Utilities, financial stocks and companies where we could not be certain that the information in the database was timely or complete were eliminated. Adjustments were also made for certain non-interest bearing liabilities. The study was structured so that an average of 30 stocks was held during the study period. Stocks with only limited liquidity were eliminated from the study. Market capitalizations were determined based on 2003 dollars. Both the number of companies in each decile as well as the number of companies in each market capitalization group fluctuated as the number of companies in the database varied during the study period.

In [6]:
# Pelo que está no livro, somentes os indicadores dos últimos 12 meses serão usados. 
# Logo, podemos descartar períodos que não são o corrente -> per_ref == 0
df.query('per_ref == 0', inplace=True)
# Remover coluna que não será mais usada no backtesting
df.drop(columns=["per_ref"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1,1,Ativo Total,278858000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01,1,Ativo Circulante,220365000.0
...,...,...,...,...,...,...,...,...,...
1038104,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,7.09.04.04,1,Part. Não Contro...,2499000.0
1038105,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,9.01.03,1,Total de Ações (...,583232411.0


Plano de contas da CVM (parte inicial "conta_id"):
- 1 -> Balance Sheet - Assets
- 2 -> Balance Sheet - Liabilities and Shareholders’ Equity
- 3 -> Income
- 4 -> Comprehensive Income
- 5 -> Changes in Equity
- 6 -> Cash Flow (Indirect Method)
- 7 -> Added Value

Logo, para usar a fórmula precisamos filtrar as empresas por:
- Liquidez mínima (o livro filtra por valor de mercado no lugar de liquidez)
- Excluir empresas do setor financeiro, seguros e utilities
- ROIC = EBIT / Invested Capital
- Invested Capital = Net Working Capital + Net Fixed Assets
- Earnings yield = EBIT / Enterprise Value*
    - Enterprise Value (EV) = market value of equity + net interest-bearing debt
    - Invested Capital (Damodaran) = PL + Dívida - Caixa (calculados pelo valor contábil)
- Desmembrando os indicadores:
    - EBIT = lucro antes dos juros e impostos -> 3.05
    - Market value of equity = núm. de ações x preço da ação -> 9.01.03
    - Net interest-bearing debt = dívida líquida
        - total_cash = df.loc["1.01.01"] + df.loc["1.01.02"]
        - total_debt = df.loc["2.01.04"] + df.loc["2.02.01"]
        - net_debt = total_debt - total_cash
    - PL -> df.loc["2.03"]

In [7]:
# Filtrar as contas que serão usadas
contas = ["1.01.01", "1.01.02", "2.01.04", "2.02.01", "2.03", "3.05", "9.01.03"]
df.query('conta_id == @contas', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,1,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,1,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...,...
41699,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,3.05,1,Resultado Antes ...,-59186000.0
41700,24600,BANCO BMG S/A,2022-02-17 18:31:34,2021-01-01,2021-12-31,9.01.03,1,Total de Ações (...,583232411.0


In [8]:
# Verificar se as contas selecionadas são fixas
df.conta_fixa.value_counts()

1    41466
0      235
Name: conta_fixa, dtype: int64

In [9]:
# Todas as empresas que contém contas não fixas nas contas selecionadas são IFs
df.query('conta_fixa == 0').cia_nome.unique()

['BCO BRADESCO S.A.', 'BCO BRASIL S.A.', 'BCO ESTADO DE S..., 'BANESTES S.A. -..., 'BCO ESTADO DO R..., ..., 'BANCO RCI BRASI..., 'Companhia de Cr..., 'BANCO BMG S/A', 'BANCO PATAGONIA..., 'PPLA PARTICIPAT...]
Length: 20
Categories (1127, object): ['2W Energia S.A.', '3A COMPANHIA SE..., '3R PETROLEUM ÓL..., '521 PARTICIPACO..., ..., 'Águas do Rio 1 ..., 'ÉVORA SA', 'ÓLEO E GÁS PART..., 'Óleos de Palma ...]

In [10]:
# Buscar demais bancos e demais IFs
procurar = "bco |banco|crédito|mercantil|seguradora|seguro|PPLA PARTICIPATIONS"
df.query('cia_nome.str.contains(@procurar, case=False)').cia_nome.unique()

['BCO BRADESCO S.A.', 'BCO AMAZONIA S.A.', 'BCO BRASIL S.A.', 'BCO ESTADO DE S..., 'BANESTES S.A. -..., ..., 'VERT-11 Companh..., 'BANCO PATAGONIA..., 'PPLA PARTICIPAT..., 'BANCO SANTANDER..., 'BRAMEX - Brasil...]
Length: 91
Categories (1127, object): ['2W Energia S.A.', '3A COMPANHIA SE..., '3R PETROLEUM ÓL..., '521 PARTICIPACO..., ..., 'Águas do Rio 1 ..., 'ÉVORA SA', 'ÓLEO E GÁS PART..., 'Óleos de Palma ...]

In [11]:
# Remover essas empresas do dataframe
df.query('~cia_nome.str.contains(@procurar, case=False)', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,1,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,1,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...,...
39392,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,3.05,1,Resultado Antes ...,241375000.0
39393,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,9.01.03,1,Total de Ações (...,216075329.0


In [12]:
# Verificar se as empresas foram removidas
df.query('cia_nome.str.contains(@procurar, case=False)').cia_nome.unique()

[], Categories (1127, object): ['2W Energia S.A.', '3A COMPANHIA SE..., '3R PETROLEUM ÓL..., '521 PARTICIPACO..., ..., 'Águas do Rio 1 ..., 'ÉVORA SA', 'ÓLEO E GÁS PART..., 'Óleos de Palma ...]

In [13]:
# Verificar novamente se as contas selecionadas são todas fixas
df.conta_fixa.value_counts()

1    39394
0        0
Name: conta_fixa, dtype: int64

In [14]:
# Remover a coluna conta fixa da base, pois não será mais usada
df.drop(columns=["conta_fixa"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...
39392,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,3.05,Resultado Antes ...,241375000.0
39393,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,9.01.03,Total de Ações (...,216075329.0


In [15]:
# Remover a descrição do código contábil para a operação futura de unstack
df.drop(columns=["conta_desc"], inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,79794000.0
...,...,...,...,...,...,...,...
39392,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,3.05,241375000.0
39393,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,9.01.03,216075329.0


In [16]:
# Unstack -> escolher as colunas que irão para o multiindex
colunas_index = df.columns[:-1].to_list()
colunas_index

['cia_id', 'cia_nome', 'doc_env', 'per_ini', 'per_fim', 'conta_id']

In [17]:
# Passar o dataframe para multiindex
df = df.set_index(colunas_index).sort_index()
df

conta_valor
cia_id cia_nome             doc_env             per_ini    per_fim    conta_id             
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31 1.01.01     4944000.0
                                                                      1.01.02    79794000.0
...                                                                                     ...
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31 9.01.03   425555795.0
                            2021-08-11 11:24:53 2018-01-01 2018-12-31 9.01.03   425555795.0

[39394 rows x 1 columns]

In [18]:
# Fazer o unstack do dataframe com base na última coluna do índice (conta_id)
df = df.unstack(level='conta_id', fill_value=0)
df

conta_valor  \
conta_id                                                                  1.01.01   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31   4944000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31   4944000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                   \
conta_id                                                                  1.01.02   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  79794000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  79794000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                   \
conta_id                                                                  2.01.04   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  61484000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  61485000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                  \
conta_id                                                                 2.02.01   
cia_id cia_nome             doc_env             per_ini    per_fim                 
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  4273000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  4273000.0   
...                                                                          ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31        0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31        0.0   

                                                                                    \
conta_id                                                                      2.03   
cia_id cia_nome             doc_env             per_ini    per_fim                   
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  142805000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  142082000.0   
...                                                                            ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31          0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31          0.0   

                                                                                   \
conta_id                                                                     3.05   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  24777000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  21093000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31 

In [19]:
# Remover o multiindex das colunas
df.columns = df.columns.droplevel(0)
df

conta_id                                                                 1.01.01  \
cia_id cia_nome             doc_env             per_ini    per_fim                 
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  4944000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  4944000.0   
...                                                                          ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31        0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31        0.0   

conta_id                                                                  1.01.02  \
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  79794000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  79794000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

conta_id                                                                  2.01.04  \
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  61484000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  61485000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

conta_id                                                                 2.02.01  \
cia_id cia_nome             doc_env             per_ini    per_fim                 
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  4273000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  4273000.0   
...                                                                          ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31        0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31        0.0   

conta_id                                                                      2.03  \
cia_id cia_nome             doc_env             per_ini    per_fim                   
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  142805000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  142082000.0   
...                                                                            ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31          0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31          0.0   

conta_id                                                                     3.05  \
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  24777000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  21093000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

conta_id                                                                   9.01.03  
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31    8856000.0  
                            2011-04-01 17:31:56 2010-01-01 2010-12-31    8856000.0  
...                                                                            ...  
503711

In [20]:
# Remover o multiindex do índice
df.reset_index(inplace=True)
df.columns.name = None
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,1.01.01,1.01.02,2.01.04,2.02.01,2.03,3.05,9.01.03
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,4944000.0,79794000.0,61484000.0,4273000.0,142805000.0,24777000.0,8856000.0
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,4944000.0,79794000.0,61485000.0,4273000.0,142082000.0,21093000.0,8856000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9461,503711,Óleos de Palma S...,2017-03-20 18:30:43,2015-01-01,2015-12-31,0.0,0.0,0.0,0.0,0.0,0.0,425555795.0
9462,503711,Óleos de Palma S...,2021-08-11 11:24:53,2018-01-01,2018-12-31,0.0,0.0,0.0,0.0,0.0,0.0,425555795.0


In [21]:
# Calcular os indicadores
df["total_cash"] = df["1.01.01"] + df["1.01.02"]
df["total_debt"] = df["2.01.04"] + df["2.02.01"]
df["net_debt"] = df["total_debt"] - df["total_cash"]
df.rename(
    columns={"2.03": "equity", "3.05": "ebit", "9.01.03": "shares_outstanding"},
    inplace=True
)
df["invested_capital"] = df["equity"] + df["net_debt"]
df["roic"] = df["ebit"] / df["invested_capital"]
df.drop(columns=["1.01.01", "1.01.02", "2.01.04", "2.02.01"], inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,net_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,-18981000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,-18980000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9461,503711,Óleos de Palma S...,2017-03-20 18:30:43,2015-01-01,2015-12-31,0.0,0.0,425555795.0,0.0,0.0,0.0,0.0,NaN
9462,503711,Óleos de Palma S...,2021-08-11 11:24:53,2018-01-01,2018-12-31,0.0,0.0,425555795.0,0.0,0.0,0.0,0.0,NaN


In [22]:
df.query('equity > 0', inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,net_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,-18981000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,-18980000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,378359000.0,92232000.0,66025325.0,1720000.0,105225000.0,103505000.0,481864000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,859939000.0,241375000.0,216075329.0,133167000.0,62406000.0,-70761000.0,789178000.0,0.305856


In [23]:
df.query('total_cash > 0', inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,net_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,-18981000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,-18980000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,378359000.0,92232000.0,66025325.0,1720000.0,105225000.0,103505000.0,481864000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,859939000.0,241375000.0,216075329.0,133167000.0,62406000.0,-70761000.0,789178000.0,0.305856


In [24]:
# Remover ebit negativos ou próximos de 0 (logo, ROIC negativo ou próximo de 0) da base,
# pois essas empresas não serão selecionadas pela fórmula
df.query('ebit >= 0.001', inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,net_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,-18981000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,-18980000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,378359000.0,92232000.0,66025325.0,1720000.0,105225000.0,103505000.0,481864000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,859939000.0,241375000.0,216075329.0,133167000.0,62406000.0,-70761000.0,789178000.0,0.305856


In [25]:
df.query('roic >= 0', inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,net_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,-18981000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,-18980000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,378359000.0,92232000.0,66025325.0,1720000.0,105225000.0,103505000.0,481864000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,859939000.0,241375000.0,216075329.0,133167000.0,62406000.0,-70761000.0,789178000.0,0.305856


In [26]:
# Remover os indicadores intermediários que não serão usados no backtesting
colunas = (
    df.columns[:5].to_list() +
    ['shares_outstanding', 'net_debt', 'ebit', 'roic']
)
df = df[colunas].copy()
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,net_debt,ebit,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,8856000.0,-18981000.0,24777000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,8856000.0,-18980000.0,21093000.0,0.171346
...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,66025325.0,103505000.0,92232000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,216075329.0,-70761000.0,241375000.0,0.305856


In [27]:
# Verificar os indicadores da Petro
# O total de ações da publicação enviada em 2011-02-25 21:33:09 foi corrigido
# na versão seguinte
df.query('cia_id == 9512')

,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,net_debt,ebit,roic
1501,9512,PETROLEO BRASILE...,2011-02-25 21:33:09,2010-01-01,2010-12-31,1.304450e+13,6.157476e+10,4.557459e+10,0.122579
1502,9512,PETROLEO BRASILE...,2011-03-04 16:04:52,2010-01-01,2010-12-31,1.304450e+10,6.157476e+10,4.557459e+10,0.122579
...,...,...,...,...,...,...,...,...,...
1518,9512,PETROLEO BRASILE...,2021-03-22 22:06:01,2020-01-01,2020-12-31,1.304450e+10,3.282680e+11,4.962100e+10,0.077603
1519,9512,PETROLEO BRASILE...,2022-02-23 22:36:50,2021-01-01,2021-12-31,1.304450e+10,2.657780e+11,2.108310e+11,0.321703


In [28]:
# Empresas sem o número de ações
# Verificar o doc. da Petro de 2018 00951220181231401.zip
df.query('shares_outstanding == 0')

,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,net_debt,ebit,roic
4093,18660,CPFL ENERGIA S.A.,2022-03-17 19:15:29,2021-01-01,2021-12-31,0.0,1.963006e+10,7.407940e+09,0.203314
9101,25526,CRUZEIRO DO SUL ...,2022-03-30 19:50:24,2021-01-01,2021-12-31,0.0,9.167400e+07,3.133410e+08,0.202749


In [29]:
# Remover essas empresas
df.query('shares_outstanding != 0', inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,net_debt,ebit,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,8856000.0,-18981000.0,24777000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,8856000.0,-18980000.0,21093000.0,0.171346
...,...,...,...,...,...,...,...,...,...
9455,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,66025325.0,103505000.0,92232000.0,0.191407
9459,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,216075329.0,-70761000.0,241375000.0,0.305856


In [30]:
df.to_csv("../data/1_financials.csv", index=False)

In [31]:
# Testar o arquivo
pd.read_csv("../data/1_financials.csv")

,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,net_debt,ebit,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,8856000.0,-18981000.0,24777000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,8856000.0,-18980000.0,21093000.0,0.171346
...,...,...,...,...,...,...,...,...,...
4056,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,66025325.0,103505000.0,92232000.0,0.191407
4057,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,216075329.0,-70761000.0,241375000.0,0.305856
